<a href="https://colab.research.google.com/github/XGHouFTW/py-cryptopredict/blob/main/StockPriceGaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
transMonth = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}

start = "2020-01-31" #from historical prices
end = "2022-01-31" #732 days
def trans(strdate):
    return datetime.datetime(int(strdate[0:4]), int(strdate[5:7]), int(strdate[8:10]))

In [ ]:
def rowinsert(row_number, df, row_value): #there does not exist a built in function for inserting a new row at a chosen index
    df1 = df[0:row_number]
    df2 = df[row_number:]
    df1.loc[row_number]=row_value
    df_result = pd.concat([df1, df2])
    df_result.index = [*range(df_result.shape[0])]

    return df_result

In [ ]:
cryptopricekey = ["BTC","ETH","DOGE"]
cryptobasepath = "/content/drive/MyDrive/ISEF/Datasets/Historical Prices/" # + cryptopricekey + "-USD.csv"
stockpricekey = ["AAPL","AMC","DJI","GME","NASDAQ","S&P","TSLA"]
stockbasepath = "/content/drive/MyDrive/ISEF/Datasets/Stonks/Raw/" # + stockpricekey + "_historical.csv"

searchkey = ["bitcoin","ethereum","dogecoin","cryptocurrency","economy","finance","politics","pandemic"]
dfPrice = pd.DataFrame()
dfAddDate = pd.read_csv("/content/drive/MyDrive/ISEF/Datasets/Historical Prices/BTC-USD.csv")
dfPrice["Date"] = dfAddDate["Date"].apply(lambda text: trans(text))

In [ ]:
for word in cryptopricekey:
  dfAdd = pd.read_csv(cryptobasepath + word + "-USD.csv")
  del dfAdd["Date"]
  dfAdd = dfAdd.rename(columns=lambda x: word + "-" + x)
  dfPrice = dfPrice.join(dfAdd) #crypto prices are continuous without fail

In [ ]:
for word in stockpricekey:
  dfAdd = pd.read_csv(stockbasepath + word + "_historical.csv")
  for i in range(len(dfAdd)):
    dfAdd.loc[i,"Date"] = trans(dfAdd.loc[i,"Date"])
  i = 0
  while (i < len(dfAdd)-1): #when stock market is closed on weekends and holidays, use the last available values
    diff = (dfAdd.loc[i+1,"Date"]-dfAdd.loc[i,"Date"]).days 
    diff -= 1 #compute streak of missing days (usually 2 for weekends)
    for j in range(diff):
      copied = dfAdd.iloc[i,:] #i denotes most recent index with available price data
      copied["Date"] = copied["Date"] + datetime.timedelta(days = j+1)
      dfAdd = rowinsert(i+j+1, dfAdd, copied) #insert copy as long as streak of missing days, with incrementing dates
    i += 1 + diff  #jump to next date with available price values
  del dfAdd["Date"]
  del dfAdd["Unnamed: 0"]
  dfAdd = dfAdd.rename(columns=lambda x: word + "-" + x)
  dfPrice = dfPrice.join(dfAdd) #merge cryptocurrency and stock prices

In [ ]:
dfPrice.to_csv("/content/drive/MyDrive/ISEF/Datasets/Stonks/Preprocessed/CryptoStockPrice.csv")